In [1]:
import numpy as np
from scipy import linalg
import os
import re
import linecache
import sys

import copy

import ase.io

import cPickle as pickle

from VASP_common_utilites import *
from nearest_neighbours_A_B import *
from parse_outcar import *
from helpers import *

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# original cell
poscar_name = 'POSCAR_skip_85'
poscar = ase.io.read(poscar_name)
init_pos_d = poscar.get_scaled_positions()
init_pos_c = poscar.get_positions()
init_latt = poscar.get_cell()
NumAtoms, AtomsLbls = parse_poscar(poscar_name)

In [4]:
# parse OUTCAR to extract later the snapshots
outcar_name = 'OUTCAR'
outcar = OUTCAR(o_name=outcar_name,p_name=poscar_name,system_name='NH4BH4',reparse=False)
# unwrap PBC
positions_direct_MD_uPBC = outcar.unwrap_PBC()

Read saved data
Done


In [5]:
# extract direct coordinates of the N, B, and H atoms
pos_N = outcar.get_direct_uPBC_3d(0)
pos_B = outcar.get_direct_uPBC_3d(1)
pos_H = outcar.get_direct_uPBC_3d(2)



[============================================================] 100.0% ...



[============================================================] 100.0% ...



[============================================================] 100.0% ...



In [6]:
# extract indexes of atoms in the SC (based on initial positions)
# an array with dimenstions if the SC is needed:
Nxyz = np.array([2,2,2])
init_N_d = init_pos_d[0:NumAtoms[0],:]
init_B_d = init_pos_d[NumAtoms[0]:NumAtoms[0:2].sum(),:]
N_UC_dict, N_UC_disp = exctrac_UC_coordinates_from_SC(init_N_d,Nxyz)
B_UC_dict, B_UC_disp = exctrac_UC_coordinates_from_SC(init_B_d,Nxyz)

In [7]:
# how mane steps to skip to account for equilibration
steps2skip = 3000
# total number of steps
total_num_steps = positions_direct_MD_uPBC.shape[0] / NumAtoms.sum()
# make array with step indeces to extract
step_distance = 200
steps = np.arange(steps2skip,total_num_steps,step_distance, dtype=int)

In [8]:
# scale the SC lattice to the UC
new_lattice_UC = np.zeros(outcar.lattice[:,:,0].shape)
for i in range(3):
    new_lattice_UC[:,i] = outcar.lattice[:,:,0][:,i] / Nxyz[i]

In [9]:
# set selective dynamics with first "immobile_num" atoms to be fixed
coord_line_idx = 8
immobile_num = 8
# make a folder to keep all the extracted SCs
cwd = os.getcwd()
try:
    os.mkdir(cwd + '/EXTRACTED_UCs')
except:
    pass
main_path = cwd + '/EXTRACTED_UCs'
# parameters to extract the complexes
min_NH = 0.1
max_NH = 1.7
ligand_num = 4
# get initial N and B positions
init_N_c = init_pos_c[0:NumAtoms[0],:]
init_B_c = init_pos_c[NumAtoms[0]:NumAtoms[0:2].sum(),:]
for idx in range(steps.shape[0]):
    step = steps[idx]
    N_direct_step = pos_N[step,:,:].T
    B_direct_step = pos_B[step,:,:].T
    H_direct_step = pos_H[step,:,:].T    
    # make a dedicated folder for the step:
    try:
        os.mkdir(main_path + '/' + str(step))
    except:
        pass
    dst = main_path + '/' + str(step)
    for SC_key in N_UC_dict.keys():
        # indeces:
        N_UC_idx = N_UC_dict[SC_key]
        B_UC_idx = B_UC_dict[SC_key]
        # the UC position in the SC
        dUC_N = N_UC_disp[SC_key]
        dUC_B = B_UC_disp[SC_key]
        # bond and other stuff dicts
        new_NH = find_all_neighbours(N_direct_step,H_direct_step,outcar.lattice[:,:,0],min_NH,max_NH)
        new_BH = find_all_neighbours(B_direct_step,H_direct_step,outcar.lattice[:,:,0],min_NH,max_NH)
        # bring the UC to the origin
        init_N_d_UC = init_N_d[N_UC_idx,:] - dUC_N
        init_B_d_UC = init_B_d[N_UC_idx,:] - dUC_B
        # scale then accordingly
        for i in range(3):
            init_N_d_UC[:,i] = init_N_d_UC[:,i] * Nxyz[i]
            init_B_d_UC[:,i] = init_B_d_UC[:,i] * Nxyz[i]
        # convert then into Cartesian coordinates of the new outcar.lattice:
        N_UC_cart = frac2cart(init_N_d_UC,new_lattice_UC)
        B_UC_cart = frac2cart(init_B_d_UC,new_lattice_UC)
        # find coordinates of the H atoms
        H_N_cart = extract_H_at_centers(new_NH,N_UC_idx,N_UC_cart)
        H_B_cart = extract_H_at_centers(new_BH,B_UC_idx,B_UC_cart)
        # append everything
        H_cart = np.append(H_N_cart, H_B_cart, axis=0)
        NB_cart = np.append(N_UC_cart,B_UC_cart, axis=0)
        UC_all_atoms_cart = np.append(NB_cart,H_cart,axis=0)
        # convert into the direct coordinates of the scaled outcar.lattice
        UC_all_atoms_direct = cart2frac(UC_all_atoms_cart, new_lattice_UC)
        UC_all_atoms_direct_PBC = implement_PBC(UC_all_atoms_direct)
        # save the results:
        system_name='4*NH4-4*BH4, unit cell ' + str(SC_key) + ' from MD snapshot at step '  + str(step)
        fname = dst + '/' + 'POSCAR_non-selective_' + str(SC_key)
        num_at = np.array([N_UC_cart.shape[0],B_UC_cart.shape[0],H_cart.shape[0]])
        txt2POSCAR = mk_text_for_POSCAR(system_name,AtomsLbls,1.0,num_at)
        write_POSCAR(fname ,UC_all_atoms_direct_PBC,new_lattice_UC,txt2POSCAR,True)
        new_fname = dst + '/' + 'POSCAR_sel_' + str(SC_key)
        impose_selective_dyn_on_part(fname,coord_line_idx,immobile_num,new_fname)